##### 20182791 권유진
##### 20182806 서동혁
##### 20182832 최준용

----------------------------------------------------------------

In [ ]:
import pandas as pd
import numpy as np
import arrow
import seaborn as sns
import matplotlib.pylab as plt
%matplotlib inline

In [ ]:
plt.rc('font', family='malgun gothic')
plt.rc('axes', unicode_minus=False)

---------------------------------------------------------------------

## 데이터 정제 작업

In [ ]:
demo = pd.read_csv('Demo.csv', encoding = 'cp949')
tran = pd.read_csv('구매내역정보.csv', encoding = 'cp949', engine = 'python')

In [ ]:
mg = pd.merge(demo, tran, on = 'ID')

In [ ]:
mg = mg.drop(mg.query('구매금액 == 0').reset_index()['index'],axis=0) # 구매금액이 0원인 것 지움.

In [ ]:
mg['평균금액'] = (mg['구매금액']/mg['구매수량']).astype(int)

# 개별 금액이 무의미해 보이는 수치들 제거
mg = mg.drop(mg.query('평균금액 < 100').reset_index()['index'],axis = 0)
mg = mg.drop(mg.query('평균금액 <= 200').reset_index()['index'], axis = 0) 
mg = mg.drop(mg[mg.평균금액.apply(lambda x: str(x)[-1]) != "0"].reset_index()['index'], axis = 0)
mg = mg.reset_index().drop('index', axis=1)
mg.head()

In [ ]:
mg.to_csv('merging_data.csv')

-------------------------------------------------------------------------

## 데이터 분석

In [ ]:
mg = pd.read_csv('merging_data.csv')

In [ ]:
mg['구매월'] = mg.구매일자.apply(lambda x: int(str(x)[4:6]))

In [ ]:
anm = pd.DataFrame()
anm['구매수량'] = mg.query('구매금액>0').groupby('상품대분류명')['구매수량'].sum()
anm['구매금액'] = mg.query('구매금액>0').groupby('상품대분류명')['구매금액'].sum()
anm['판매량 대비 매출'] = [round(anm['구매금액'][i]/anm['구매수량'][i],1) for i in range(len(anm['구매수량']))]
anm['판매금액비율'] = list(map(lambda x: round(anm.구매금액[x]/anm.구매금액.sum()*100,1), range(len(anm.구매금액))))
anm = anm.reset_index()
anm

In [ ]:
plt.pie(anm.구매수량, labels = anm.상품대분류명, shadow=True, autopct='%1.1f%%')
plt.title('상품대분류별 판매금액비율')
plt.show()

In [ ]:
sns.catplot(y='판매량 대비 매출', x='상품대분류명', data = anm, kind = 'bar')
plt.title('대분류별 판매효율')

plt.show()

In [ ]:
refund = pd.DataFrame()
refund['환불수량'] = mg.query('구매수량<0').groupby('상품대분류명')['구매수량'].sum().apply(lambda x: -int(x))
refund['환불금액'] = mg.query('구매수량<0').groupby('상품대분류명')['구매금액'].sum().apply(lambda x: -int(x))
refund['판매량 대비 환불량'] = [round(refund['환불수량'][i]/anm['구매수량'][i]*100,1) for i in range(len(refund['환불수량']))]
refund['금액 비율'] = [round(refund['환불금액'][i]/anm['구매금액'][i]*100,1) for i in range(len(refund['환불수량']))]
refund=refund.reset_index()
refund

In [ ]:
sns.catplot(x='상품대분류명', y='환불금액', kind = 'bar', data = refund)
plt.title('상품대분류별 판매량 대비 환불량')
plt.show()

In [ ]:
result = pd.DataFrame()
result['최종판매량'] = mg.groupby('상품대분류명')['구매수량'].sum()
result['최종판매금액'] = mg.groupby('상품대분류명')['구매금액'].sum()
result['매출비율'] = list(map(lambda x: round(result.최종판매금액[x]/result.최종판매금액.sum()*100,1), range(len(result.최종판매금액))))
result = result.reset_index()
result

In [ ]:
plt.pie(result.매출비율, labels = result.상품대분류명, shadow=True,autopct='%1.1f%%')
plt.title('총 매출 비율')
plt.show()

In [ ]:
Mb1 = mg.groupby('구매월')['구매수량'].sum().reset_index()
sns.relplot(x='구매월', y='구매수량', kind='line', data = Mb1, color = 'black', marker='o', mfc='red')
plt.title('월별 총구매수량')
plt.xticks(range(1,13))
plt.ylim(5000,)
plt.grid()

plt.show()

In [ ]:
Mb2 = mg.groupby(['구매월', '상품대분류명'])['구매수량'].sum().reset_index()
sns.relplot(x='구매월', y='구매수량', kind='line', data = Mb2, hue='상품대분류명', marker='o')
plt.title('상품대분류별 총구매수량 변화')
plt.xticks(range(1,13))
plt.grid()

plt.show()

In [ ]:
Mb2 = mg.query('상품대분류명 in ["가전제품", "레포츠","명품","생활잡화","가구"]').groupby(['구매월', '상품대분류명'])['구매수량'].sum().reset_index()
sns.relplot(x='구매월', y='구매수량', kind='line', data = Mb2, hue='상품대분류명', marker='o')
plt.title('상품대분류별 총구매수량 변화')
plt.xticks(range(1,13))
plt.grid()

plt.show()

In [ ]:
Mb2 = mg.groupby(['구매월', '상품대분류명'])['구매금액'].sum().reset_index()
sns.relplot(x='구매월', y='구매금액', kind='line', data = Mb2, hue='상품대분류명', marker='o')
plt.title('상품대분류별 총구매금액 변화')
plt.xticks(range(1,13))
plt.grid()

plt.show()

In [ ]:
Mb2 = mg.query('상품대분류명 in ["가구","레포츠","명품","생활잡화","식품"]').groupby(['구매월', '상품대분류명'])['구매금액'].sum().reset_index()
sns.relplot(x='구매월', y='구매금액', kind='line', data = Mb2, hue='상품대분류명', marker='o')
plt.title('상품대분류별 총구매금액 변화')
plt.xticks(range(1,13))
plt.grid()

plt.show()

In [ ]:
Mb3 = mg.query('상품대분류명=="가전제품"').groupby(['구매월', '상품중분류명'])['구매수량'].sum().reset_index()
sns.relplot(x='구매월', y='구매수량', kind='line', data = Mb3, hue='상품중분류명', marker = 'o')
plt.title('가전제품 중분류별 구매수량 변화')
plt.xticks(range(1,13))
plt.grid()

plt.show()

In [ ]:
Mb3_1 = mg.query('상품중분류명=="휴대폰/태블릿" or 상품중분류명=="PC/노트북/프린터/카메라"').groupby(['구매월', '상품중분류명'])['구매수량'].sum().reset_index()
sns.relplot(x='구매월', y='구매수량', kind='line', data = Mb3_1, hue='상품중분류명', marker = 'o')
plt.title('가전제품 중분류별 구매수량 변화')
plt.xticks(range(1,13))
plt.grid()

plt.show()

In [ ]:
mb3 = pd.merge(Mb3.query('구매월 == 7'),Mb3.query('구매월 == 8'),on='상품중분류명')[['상품중분류명','구매수량_x','구매수량_y']].rename(columns={'구매수량_x':'7월 구매수량','구매수량_y':'8월 구매수량'})
mb3['x배 증가'] = round(mb3['8월 구매수량']/mb3['7월 구매수량'],1)
mb3

In [ ]:
Mb3 = mg.query('상품대분류명=="가전제품"').groupby(['구매월', '상품중분류명'])['구매금액'].sum().reset_index()
sns.relplot(x='구매월', y='구매금액', kind='line', data = Mb3, hue='상품중분류명', marker = 'o')
plt.title('가전제품 중분류별 구매금액 변화')
plt.xticks(range(1,13))
plt.grid()

plt.show()

In [ ]:
Mb3 = mg.query('상품중분류명=="휴대폰/태블릿" or 상품중분류명=="PC/노트북/프린터/카메라" or 상품중분류명=="컴퓨터주변기기"').groupby(['구매월', '상품중분류명'])['구매금액'].sum().reset_index()
sns.relplot(x='구매월', y='구매금액', kind='line', data = Mb3, hue='상품중분류명', marker = 'o')
plt.title('가전제품 중분류별 구매금액 변화')
plt.xticks(range(1,13))
plt.grid()

plt.show()